# Impact of GDP per capita on employment in agriculture over the past 30 years  

In this project we will explore data regarding GDP per capita and the employment in the agricultural sector. We will use data over the past 30 years from countries all over the world provided via the API from the WorldBank. To read further into our idea of this project we would like to refer to our ReadMe-file which is provided in our GitHub-repository. Lets get started:

Imports and set magics:

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots 
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
from functools import reduce

# import the API for WorldBank-data
from pandas_datareader import wb

# Read data

Our first step is to import data using the API provided by the WorldBank. We found the following data and data-codes using the Website of the WorldBank to provide us with different measures for countries worldwide over the past decades:
#
-Arable land in hectares --> *AG.LND.ARBL.HA*\
-Labor force --> *SL.TLF.TOTL.IN*\
-GDP per capita in USD --> *NY.GDP.PCAP.CD*\
-Employment in agriculture in % of total empoyment --> *SL.AGR.EMPL.ZS*

To read further into our source of data you can use this link right here to get to the website of the WorldBank. Put the italic codes into the URL to acess the specific sites:
https://data.worldbank.org/indicator/

In [2]:
# Setting up list of ISO-codes of countries where we are going to import data from using the API
countries_ISO_list = ['ABW', 'AFG', 'AGO', 'ALB', 'AND', 'ARE', 'ARG', 'ARM', 'ASM', 'AUS', 'AUT', 
                      'AZE', 'BDI', 'BEL', 'BEN', 'BFA', 'BGD', 'BGR', 'BHR', 'BHS', 'BIH', 'BLR', 
                      'BLZ', 'BMU', 'BOL', 'BRA', 'BRB', 'BRN', 'BTN', 'BWA', 'CAF', 'CAN', 'CHE', 
                      'CHL', 'CHN', 'CIV', 'CMR', 'COD', 'COG', 'COL', 'COM', 'CPV', 'CRI', 'CUB', 
                      'CUW', 'CYM', 'CYP', 'CZE', 'DEU', 'DJI', 'DMA', 'DNK', 'DOM', 'DZA', 'ECU', 
                      'EGY', 'ERI', 'ESP', 'EST', 'ETH', 'FIN', 'FJI', 'FRA', 'FRO', 'GAB', 'GBR', 
                      'GEO', 'GHA', 'GIB', 'GIN', 'GMB', 'GNB', 'GNQ', 'GRC', 'GRD', 'GRL', 'GTM', 
                      'GUY', 'HKG', 'HND', 'HRV', 'HTI', 'HUN', 'IDN', 'IMN', 'IND', 'IRL', 'IRN', 
                      'IRQ', 'ISL', 'ISR', 'ITA', 'JAM', 'JOR', 'JPN', 'KAZ', 'KEN', 'KGZ', 'KHM', 
                      'KIR', 'KNA', 'KOR', 'KWT', 'LBN', 'LBR', 'LBY', 'LCA', 'LIE', 'LKA', 'LSO', 
                      'LTU', 'LUX', 'LVA', 'MAC', 'MAF', 'MAR', 'MCO', 'MDA', 'MDG', 'MDV', 'MEX', 
                      'MKD', 'MLI', 'MLT', 'MMR', 'MNE', 'MNG', 'MOZ', 'MRT', 'MUS', 'MWI', 'MYS', 
                      'NAM', 'NCL', 'NER', 'NGA', 'NIC', 'NLD', 'NOR', 'NPL', 'NRU', 'NZL', 'OMN', 
                      'PAK', 'PAN', 'PER', 'PHL', 'PLW', 'PNG', 'POL', 'PRI', 'PRK', 'PRT', 'PRY', 
                      'PSE', 'PYF', 'QAT', 'ROU', 'RUS', 'RWA', 'SAU', 'SDN', 'SEN', 'SGP', 'SLE', 
                      'SLV', 'SMR', 'SOM', 'SRB', 'SSD', 'STP', 'SUR', 'SVK', 'SVN', 'SWE', 'SWZ', 
                      'SXM', 'SYC', 'SYR', 'TCA', 'TCD', 'TGO', 'THA', 'TJK', 'TKM', 'TLS', 'TON', 
                      'TTO', 'TUN', 'TUR', 'TUV', 'TZA', 'UGA', 'UKR', 'URY', 'USA', 'UZB', 'VCT', 
                      'VEN', 'VGB', 'VIR', 'VNM', 'VUT', 'WSM', 'YEM', 'ZAF', 'ZMB', 'ZWE']

# Importing the different DataFrames from the WorldBank using the API from the lecture and resetting the index for each DataFrame
wb_arable_land = wb.download(indicator='AG.LND.ARBL.HA', country= countries_ISO_list, start=1989, end=2019)
wb_arable_land = wb_arable_land.reset_index()
wb_labor_force = wb.download(indicator='SL.TLF.TOTL.IN', country= countries_ISO_list, start=1989, end=2019)
wb_labor_force = wb_labor_force.reset_index()
wb_emp_ag = wb.download(indicator='SL.AGR.EMPL.ZS', country= countries_ISO_list, start=1989, end=2019)
wb_emp_ag = wb_emp_ag.reset_index()
wb_gdp_cap = wb.download(indicator='NY.GDP.PCAP.CD', country= countries_ISO_list, start=1989, end=2019)
wb_gdp_cap = wb_gdp_cap.reset_index()
wb_cereal_yield = wb.download(indicator='AG.YLD.CREL.KG', country= countries_ISO_list, start=1989, end=2019)
wb_cereal_yield = wb_cereal_yield.reset_index()

# Clean and merge data 

Our second step is then to clean up the dataframes provided by the API and to merge them together into a bigger one. We drop rows with non-existing values, round values and add units for better display. An overview of the final DataFrame is shown below.

In [9]:
# changing the data-type of the 'year'-column to an integer to allow merging and plotting later 
wb_arable_land.year = wb_arable_land.year.astype(int)
wb_labor_force.year = wb_labor_force.year.astype(int)
wb_emp_ag.year = wb_emp_ag.year.astype(int)
wb_gdp_cap.year = wb_gdp_cap.year.astype(int)
wb_cereal_yield.year = wb_cereal_yield.year.astype(int)

# Merging all the DataFrames into one big one
data_frames_list = [wb_arable_land, wb_labor_force, wb_emp_ag, wb_gdp_cap, wb_cereal_yield]
data_frames_combined = reduce(lambda left, right: pd.merge(left, right, on = ['country', 'year'], how = 'inner'), data_frames_list)

# Renaming the columns
namedict = {
            'AG.LND.ARBL.HA':'Arable_Land',
            'SL.TLF.TOTL.IN':'Labor_Force',
            'SL.AGR.EMPL.ZS':'Empl_in_agri_in_%_of_all_empl',
            'NY.GDP.PCAP.CD':'GDP_per_capita',
            'AG.YLD.CREL.KG' : 'Cereal_Yield_kg_per_ha'}
data_frames_combined.rename(columns=namedict, inplace=True)

# Dropping all rows that contain empty values (NAN) and resetting index
data_frames_combined.dropna(inplace=True)
data_frames_combined = data_frames_combined.reset_index(drop=True)

# Rounding the floats in columns 'Empl_in_agri_in_%_of_all_empl' and 'GDP_per_capita' 
data_frames_combined['Empl_in_agri_in_%_of_all_empl'] = data_frames_combined['Empl_in_agri_in_%_of_all_empl'].round(2)
data_frames_combined['GDP_per_capita'] = data_frames_combined['GDP_per_capita'].round(2)

# Converting the values in the columns into their respective units for a better display
data_frames_combined['Empl_in_agri_in_%_of_all_empl'] = data_frames_combined['Empl_in_agri_in_%_of_all_empl'].apply(lambda x: '{:.2f}%'.format(x*1))
data_frames_combined['GDP_per_capita'] = data_frames_combined['GDP_per_capita'].apply(lambda x: '${:.2f}'.format(x))
data_frames_combined['Arable_Land'] = data_frames_combined['Arable_Land'].apply(lambda x: '{:.0f} ha'.format(x))
data_frames_combined.Labor_Force = data_frames_combined.Labor_Force.astype(int)

# Display the created DataFrame so far
data_frames_combined

,country,year,Arable_Land,Labor_Force,Empl_in_agri_in_%_of_all_empl,GDP_per_capita,Cereal_Yield_kg_per_ha
0,Afghanistan,2019,7788000 ha,9991682,42.50%,$500.52,2113.4
1,Afghanistan,2018,7794000 ha,9613728,43.13%,$502.06,2162.0
2,Afghanistan,2017,7699000 ha,9254594,43.99%,$530.15,2022.5
3,Afghanistan,2016,7729000 ha,8913938,45.81%,$520.25,1980.4
4,Afghanistan,2015,7765000 ha,8630723,47.07%,$592.48,2132.2
...,...,...,...,...,...,...,...
4752,Zimbabwe,1995,3110000 ha,3958602,60.92%,$646.83,548.2
4753,Zimbabwe,1994,3100000 ha,3871382,61.08%,$634.58,1134.5
4754,Zimbabwe,1993,3030000 ha,3824653,61.40%,$608.05,1502.0
4755,Zimbabwe,1992,2990000 ha,3761641,61.47%,$634.45,412.1


## Exploring GDP per capita and employment in the agricultural sector for different countries

In order to explore how GDP per capita and the percentage of people employed in the agricultural sector have changed over the past 30 years in different countries worldwide we decided to create an interactive plot. An explaination of the plot is provided down below. 

**Interactive plot** :

In [10]:
# Define the countries list
countries_list = data_frames_combined['country'].unique().tolist()

# Create the combined plot function
def plot_combined(country):
    # Filter the data for the selected country
    country_data = data_frames_combined[data_frames_combined['country'] == country]

    # Create the first subplot for GDP per capita and employment in agriculture
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, specs=[[{"secondary_y": True}],[{}]])

    # Add GDP per capita trace (left y-axis)
    fig.add_trace(go.Scatter(x=country_data['year'], y=country_data['GDP_per_capita'],
                             mode='lines+markers', name='GDP per capita'), row=1, col=1, secondary_y=False)

    # Add percentage of people working in agriculture trace (right y-axis)
    fig.add_trace(go.Scatter(x=country_data['year'], y=country_data['Empl_in_agri_in_%_of_all_empl'],
                             mode='lines+markers', name='Employment in agriculture (%)',
                             yaxis='y2'), row=1, col=1, secondary_y=True)

    # Set the plot title and axis labels
    fig.update_layout(
        title=f'GDP per capita, employment in agriculture, and cereal yield for {country}',
        xaxis_title='Year',
        yaxis_title='GDP per capita in USD',
        yaxis2=dict(
            title='Employment in agriculture (%)',
            overlaying='y',
            side='right'
        )
    )

    # Create the second subplot for cereal yield
    fig.add_trace(go.Scatter(x=country_data['year'], y=country_data['Cereal_Yield_kg_per_ha'],
                             mode='lines+markers', line=dict(color='green'), name='Cereal Yield'),
                  row=2, col=1)

    # Set the plot title and axis labels
    fig.update_yaxes(title_text="Cereal Yield (kg per ha)", row=2, col=1)

    # Set the plot layout
    fig.update_layout(height=800, width=800, showlegend=True, template="plotly_white")

    # Show the plot
    fig.show()

# Create a dropdown menu to select a country
interact(plot_combined, country=widgets.Dropdown(options=countries_list, value=countries_list[0], description='Country:'))

interactive(children=(Dropdown(description='Country:', options=('Afghanistan', 'Angola', 'Albania', 'United Ar…

<function __main__.plot_combined(country)>

<span style="color:red">DISCLAIMER: DUE TO THE AMOUNT OF DATA USED HERE IT MIGHT BE POSSIBLE THAT THE INTERACTIVE PLOT ABOVE DOES NOT AUTO-DISPLAY PROPERLY AND A COUNTRY MUST FIRST BE MANUALLY SELECTED!</span>


EXPLANATION OF THE PLOT: The first plot allows the reader to get an idea of the development of GDP per capita and the percentage of workers employed in the agricultural sector over the past 30 years in the different countries of the world. The left Y-axis denotes GDP per capita in USD, the right Y-axis denotes the percentage of workers employed in agriculture and the X-axis sets the year. The second plot displays the crop yields over the same timeframe. 

In general it can be noted (espesially for already developed or developing nations) that the richer a country gets on a per capita basis a smaller percentage of the working force is employed in agriculture. At the same time crop yields ramain stable or increase showing increasing productivity of the farmed lands. This might be due to:
- Increased mechanisation of farming (which should increase productivity)
- Improved agro-chemicals and crop varieties 
- Acess to better information (for example farming practices or weather)
- More attractive job opportunities outside of the agricultural sector that draw workers away from farming 

# Further Analysis of the biggest agricultural producers 

We are now continuing with a subset of our DataFrame containing the 10 countries with the largest area of arable land. In this Dataframe we will try to find a relationship between the influence of GDP per capita on the amount of arable land farmed per single worker in agriculture. 


First we will extract the 10 countries with the most arable land from our DataFrame above and then remove all other countries (the cut down Dataframe is depicted below):

In [11]:
# a. First change the units of the columns back to floats and integers to allow for operations on those values
data_frames_combined['Empl_in_agri_in_%_of_all_empl'] = data_frames_combined['Empl_in_agri_in_%_of_all_empl'].str.replace('%','').astype(float)
data_frames_combined['GDP_per_capita'] = data_frames_combined['GDP_per_capita'].str.replace('$','').astype(float)
data_frames_combined['Arable_Land'] = data_frames_combined['Arable_Land'].str.replace(' ha','').astype(int)

C:\Users\luke-\AppData\Local\Temp\ipykernel_17308\425018561.py:3: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.



In [12]:
# b. Creating a list of the 10 biggest countries by area of arable land 
df_sorted = data_frames_combined.sort_values('year', ascending=False)   # Sort big dataframe by year 
grouped = df_sorted.groupby('country')                                  # Group by country
latest_year = grouped.head(1)                                           # Keep rows for most recent year for each country
largest_countries = latest_year.nlargest(10, 'Arable_Land')             # Get the 10 largest in a DataFrame
country_column = largest_countries['country']                           # Get the country column as a Series object
top_ten_country_list = country_column.tolist()                          # Convert into a Python list

# Printing which countries we are going to keep 
print(f"The 10 biggest countries by area of arable land that remain in the DataFrame are:\n {top_ten_country_list}")

# Cutting down the big WorldBank-DataFrame to the 10 countries with the largest area of arable land
data_frame_ten_biggest = data_frames_combined[data_frames_combined['country'].isin(top_ten_country_list)]
data_frame_ten_biggest = data_frame_ten_biggest.reset_index(drop=True)

# Displaying the new cut down Dataframe
data_frame_ten_biggest

The 10 biggest countries by area of arable land that remain in the DataFrame are:
 ['United States', 'India', 'Russian Federation', 'China', 'Brazil', 'Canada', 'Nigeria', 'Ukraine', 'Argentina', 'Australia']


,country,year,Arable_Land,Labor_Force,Empl_in_agri_in_%_of_all_empl,GDP_per_capita,Cereal_Yield_kg_per_ha
0,Argentina,2019,32632760,20607477,0.06,9963.67,5474.8
1,Argentina,2018,32632760,20103008,0.10,11795.16,4671.4
2,Argentina,2017,33984900,19580710,0.06,14613.04,5404.6
3,Argentina,2016,35336500,19404277,0.13,12790.24,5203.4
4,Argentina,2015,36688100,19229475,0.26,13789.06,4779.1
...,...,...,...,...,...,...,...
283,United States,1995,181839000,135975699,1.83,28690.88,4644.8
284,United States,1994,181939000,134076656,1.86,27694.85,5560.0
285,United States,1993,182748000,131792391,1.89,26387.29,4298.8
286,United States,1992,184080000,130387362,1.91,25418.99,5357.8


To get to the relationship between the GDP per capita and the area of arable land farmed by a single person working in the agriculture sector we will need to compute some new columns out of our original WorldBank-data due to the fact that not all the data was available in the way we would need it.

- We will have to compute the absolute amount of people working in the agriculture sector per country from the columns of the labor force and the percentage of people employed in agriculture. 

- We will also calculate a new column of the arable land divided by the people working in the agriculture sector. This then gives us an indicator on how much area of arable land one single worker can handle.

In [13]:
# Create new column of absolute amount of people working in the agriculture sector
data_frame_ten_biggest['People_in_agri'] = (data_frame_ten_biggest['Labor_Force'] * (data_frame_ten_biggest['Empl_in_agri_in_%_of_all_empl'] / 100)).astype(int)

# Create a new column of the arable land divided by the people working in the agriculture sector
data_frame_ten_biggest['Land_per_agri_worker'] = data_frame_ten_biggest['Arable_Land'] / data_frame_ten_biggest['People_in_agri'] 

# Display the Dataframe  
data_frame_ten_biggest

,country,year,Arable_Land,Labor_Force,Empl_in_agri_in_%_of_all_empl,GDP_per_capita,Cereal_Yield_kg_per_ha,People_in_agri,Land_per_agri_worker
0,Argentina,2019,32632760,20607477,0.06,9963.67,5474.8,12364,2639.336784
1,Argentina,2018,32632760,20103008,0.10,11795.16,4671.4,20103,1623.278118
2,Argentina,2017,33984900,19580710,0.06,14613.04,5404.6,11748,2892.824311
3,Argentina,2016,35336500,19404277,0.13,12790.24,5203.4,25225,1400.852329
4,Argentina,2015,36688100,19229475,0.26,13789.06,4779.1,49996,733.820706
...,...,...,...,...,...,...,...,...,...
283,United States,1995,181839000,135975699,1.83,28690.88,4644.8,2488355,73.075988
284,United States,1994,181939000,134076656,1.86,27694.85,5560.0,2493825,72.955801
285,United States,1993,182748000,131792391,1.89,26387.29,4298.8,2490876,73.366960
286,United States,1992,184080000,130387362,1.91,25418.99,5357.8,2490398,73.915896


**Interactive plot** :

In [14]:
def plot_gdp_per_capita_and_land_per_agri_worker(country):

    country_data_new = data_frame_ten_biggest[data_frame_ten_biggest['country'] == country]

    x = country_data_new['GDP_per_capita']
    y = country_data_new['Land_per_agri_worker']
    m, b = np.polyfit(x, y, 1)
    line = m * x + b

    fig2 = go.Figure()

    fig2.add_trace(go.Scatter(
        x=country_data_new['GDP_per_capita'], 
        y=country_data_new['Land_per_agri_worker'], 
        mode='markers',
        marker=dict(
            size=10,
            color=country_data_new['year'], 
            colorscale='plasma', 
            opacity=0.8,
            colorbar=dict(
                title='Year'
            )
        ),
        text=country_data_new['year'],
        name=country
    ))

    fig2.add_trace(go.Scatter(x=x, y=line, mode='lines', name='Line of best fit'))

    fig2.update_layout(
        title=f'GDP per capita and land per agri worker {country}',
        xaxis_title='GDP per capita in USD',
        yaxis_title='Land per agri worker in ha',
        yaxis2=dict(
            title='Land per agri worker',
            overlaying='y',
            side='right'
        )
    )

    fig2.show() 


interact(plot_gdp_per_capita_and_land_per_agri_worker, country=widgets.Dropdown(options=top_ten_country_list, value=top_ten_country_list[3], description='Country:'))

interactive(children=(Dropdown(description='Country:', index=3, options=('United States', 'India', 'Russian Fe…

<function __main__.plot_gdp_per_capita_and_land_per_agri_worker(country)>

EXPLANATION OF THE PLOT: This plot allows the reader to get an idea how the area farmed by one single agricultural worker changed in each of the ten biggest agri-producers over the past 30 years in light of the change in GDP per capita. The Y-axis depicts the hectars one single worker farmes in hectars, the X-axis depicts the GDP per capita in USD and the colormap brings the time dimension into the plot. A Regression-line of best fit was added to show the relationship between the X- and Y-variable.

Interestingly some countries do show an almost liniar positiv relationship between GDP per capita and the amount of land farmed by a single worker. Most notably China, which had massive economic growth over the last decades, does show this. Other countries like Australia, Ukraine and Canada also show positive relationships while some countries like the United States and India do not produce clear pictures with declining lines of best fit.  

# Conclusion

IN GENERAL we can conclude:

- For the most part and most countries an increase in GDP per capita goes hand in hand with a decrease in employment in agriculture. Meanwhile, the crop yield increases despite this fact. This leads to the conclusion that it is factors like more mechanization and the use of improved agro-chemicals that allow for higher productivity in the agriculture sector. 

- Our analysis of the ten biggest countries measured by arable land showed that the relationship between GDP per capita and the amount of land farmed by a single worker is mostly of positive nature. Especially the rapid development of China seems to have played a role in shaping the relationship seen here. 

- This however is a very broad analysis. The conclusions drawn here are not very concrete and other factors need to be evaluated when reseraching these kinds of relationships on a country by country basis (for example changing local laws regulations might have an impact as well).